# **Proyecto 1 - Data Science**
## **Limpieza de datos**

In [1]:
import pandas as pd
import unicodedata
import re

df = pd.read_csv("data_unificada.csv")


### **Crear carpeta destino para alojar dataset limpio**

In [2]:

import os
import shutil

carpeta_destino = 'output'
carpeta_destino_csv = os.path.join(carpeta_destino, 'csv')

# Crear la carpeta si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
    print(f"Carpeta '{carpeta_destino}' creada.")

### **Inspección inicial del estado de los datos**

In [3]:
# Tamaño y columnas
print("Dimensiones:", df.shape)
print("Columnas:", df.columns.tolist())

# Revisar nulos
print("Valores nulos por columna:\n", df.isnull().sum())

# Tipos de datos
print("Tipos de datos:\n", df.dtypes)

df.sample(5)


Dimensiones: (6584, 17)
Columnas: ['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']
Valores nulos por columna:
 CODIGO              0
DISTRITO            0
DEPARTAMENTO        0
MUNICIPIO           0
ESTABLECIMIENTO     0
DIRECCION           2
TELEFONO           45
SUPERVISOR          0
DIRECTOR           23
NIVEL               0
SECTOR              0
AREA                0
STATUS              0
MODALIDAD           0
JORNADA             0
PLAN                0
DEPARTAMENTAL       0
dtype: int64
Tipos de datos:
 CODIGO             object
DISTRITO           object
DEPARTAMENTO       object
MUNICIPIO          object
ESTABLECIMIENTO    object
DIRECCION          object
TELEFONO           object
SUPERVISOR         object
DIRECTOR           object
NIVEL              object
SECTOR             object
AREA               object
STATUS        

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
5637,12-07-0216-46,12-073,SAN MARCOS,TACANA,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA,CANTÓN CRUZ DE BARRANCAS,40183677.0,DUBEN HUMBERTO CIFUENTES GONZALEZ,ROXANA VERENISSE GONZÁLEZ RECINOS,DIVERSIFICADO,OFICIAL,RURAL,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),SAN MARCOS
4835,14-01-0112-46,14-001,QUICHE,SANTA CRUZ DEL QUICHE,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,"20 CALLE FINAL ZONA 4, COLONIA LOS CELAJES",77554630,JOSE OSCAR TIPAZ VELASQUEZ,JUSTO RUFINO PEREIRA GONZÁLEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUICHÉ
4989,14-20-0184-46,14-054,QUICHE,IXCAN,COLEGIO AMERICANO MAYA QUICHE,"LOTE 08 COLONIA BUENA VISTA, ENTRADA A SAN FRA...",77557616,JERONIMO COC JUAREZ,MAYLING PATRICIA PINEDA URIZAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (FIN DE SEMANA),QUICHÉ NORTE
5199,11-04-0033-46,11-017,RETALHULEU,SAN MARTIN ZAPOTITLAN,INSTITUTO MUNICIPAL DE EDUCACIÓN MEDIA Y BACHI...,0 AVENIDA ZONA 1,53271115.0,MIGUEL ANGEL ARMAS ROCHA,HÉCTOR ANÍBAL MALDONADO ALVARADO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (FIN DE SEMANA),RETALHULEU
1424,00-12-0140-46,01-211,CIUDAD CAPITAL,ZONA 12,"COLEGIO PARROQUIAL ""PADRE ELOY SUAREZ COBIAN""",26 CALLE 7-10,24770712,ADRIANA FABIOLA BOLAÑOS ESTRADA,MASSIEL ZAVALA CASTRO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA SUR


### **Limpiar columnas tipo texto: eliminar espacios extremos y convertir a mayúsculas**

In [4]:

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip().str.upper()


En este paso se estandarizó todo el texto de las columnas tipo cadena. Esto se realizó con el propósito de evitar inconsistencias causadas por diferencias en mayúsculas y minúsculas, y por espacios innecesarios al inicio o final de los campos. Esta limpieza permite mejorar la detección de duplicados, corregir errores ortográficos y facilitar futuras agrupaciones.

## **CÓDIGO**

### **Ver ejemplos distintos de códigos (únicos)**

In [5]:
df["CODIGO"].sample(20, random_state=1)

5161    11-01-1406-46
2805    01-14-0187-46
3327    13-04-0112-46
3531    18-01-0320-46
6333    10-08-0026-46
3821    21-01-0417-46
4152    22-14-0080-46
1851    05-01-0459-46
6414    10-15-0040-46
4523    09-01-0384-46
5442    03-09-0056-46
1793    05-01-0253-46
1584    00-18-0223-46
1691    02-01-0062-46
5891    12-29-0029-46
3172    01-17-0315-46
1988    05-05-0077-46
4179    22-16-0031-46
2851    01-15-0119-46
552     04-03-0260-46
Name: CODIGO, dtype: object

### **Ver longitud del identificador de cada establecimiento**

In [6]:

df["LARGO_CODIGO"] = df["CODIGO"].astype(str).str.len()
df["LARGO_CODIGO"].value_counts().sort_index()



LARGO_CODIGO
13    6584
Name: count, dtype: int64

Se evaluó la estructura de los códigos institucionales para garantizar su consistencia. Esto permitió detectar posibles errores de codificación o registros atípicos. No existen variaciones en el formato ni en la cantidad de dígitos en cada código.

In [7]:
df = df.drop("LARGO_CODIGO", axis=1)

## **DISTRITO**

### **Ver ejemplos distintos de códigos de cada distrito (únicos)**

In [8]:
df["DISTRITO"].sample(20, random_state=1)

5161    11-017
2805    01-411
3327    13-012
3531    18-008
6333    10-009
3821    21-004
4152    22-023
1851    05-033
6414    10-021
4523    09-006
5442    03-006
1793    05-033
1584    01-403
1691    02-021
5891    12-053
3172    01-641
1988    05-031
4179    22-026
2851    01-214
552     04-033
Name: DISTRITO, dtype: object

### **Ver longitud del identificador de cada distrito**

In [9]:
df["LARGO_DISTRITO"] = df["DISTRITO"].astype(str).str.len()
df["LARGO_DISTRITO"].value_counts().sort_index()


LARGO_DISTRITO
6    6584
Name: count, dtype: int64

Se evaluó la estructura de los identificadores de cada distrito para garantizar su consistencia. Esto permitió detectar posibles errores de codificación o registros atípicos. No existen variaciones en el formato ni en la cantidad de dígitos en cada identificador del distrito

In [10]:
df = df.drop("LARGO_DISTRITO", axis=1)

## **DEPARTAMENTO**

### **Eliminar espacios inconsistentes y cualquier acentuación**

In [11]:
def limpiar_departamento(nombre):
    nombre = unicodedata.normalize("NFKD", nombre)
    nombre = ''.join(c for c in nombre if not unicodedata.combining(c))
    return nombre

df["DEPARTAMENTO"] = df["DEPARTAMENTO"].apply(limpiar_departamento)

### **Obtener los valores únicos**

In [12]:
print("Departamentos únicos encontrados:")
print(df["DEPARTAMENTO"].sort_values().unique())

Departamentos únicos encontrados:
['ALTA VERAPAZ' 'BAJA VERAPAZ' 'CHIMALTENANGO' 'CHIQUIMULA'
 'CIUDAD CAPITAL' 'EL PROGRESO' 'ESCUINTLA' 'GUATEMALA' 'HUEHUETENANGO'
 'IZABAL' 'JALAPA' 'JUTIAPA' 'PETEN' 'QUETZALTENANGO' 'QUICHE'
 'RETALHULEU' 'SACATEPEQUEZ' 'SAN MARCOS' 'SANTA ROSA' 'SOLOLA'
 'SUCHITEPEQUEZ' 'TOTONICAPAN' 'ZACAPA']


### **Validar contra la lista de departamentos**

In [13]:

departamentos_oficiales = [
    "ALTA VERAPAZ", "BAJA VERAPAZ", "CHIMALTENANGO", "CHIQUIMULA",
    "EL PROGRESO", "ESCUINTLA", "GUATEMALA", "HUEHUETENANGO",
    "IZABAL", "JALAPA", "JUTIAPA", "PETEN", "QUETZALTENANGO",
    "QUICHE", "RETALHULEU", "SACATEPEQUEZ", "SAN MARCOS",
    "SANTA ROSA", "SOLOLA", "SUCHITEPEQUEZ", "TOTONICAPAN", "ZACAPA",
    "CIUDAD CAPITAL" 
]

departamentos_encontrados = set(df["DEPARTAMENTO"].unique())
departamentos_invalidos = departamentos_encontrados - set(departamentos_oficiales)

print(" Departamentos no oficiales encontrados:")
print(departamentos_invalidos)


 Departamentos no oficiales encontrados:
set()


Se normalizó el campo `DEPARTAMENTO` mediante la conversión a mayúsculas, eliminación de espacios y tildes. Posteriormente, se comparó contra la lista oficial de departamentos de Guatemala para detectar valores inválidos. Los errores se corrigen si se deben a variantes tipográficas; si no, se analizan individualmente.

## **MUNICIPIO**

### **Eliminar espacios inconsistentes y cualquier acentuación**

In [14]:
def limpiar_municipio(nombre):
    nombre = unicodedata.normalize("NFKD", nombre)
    nombre = ''.join(c for c in nombre if not unicodedata.combining(c))
    nombre = " ".join(nombre.split())  
    return nombre

df["MUNICIPIO"] = df["MUNICIPIO"].apply(limpiar_municipio)

### **Revisar valores únicos**

In [15]:
print("Cantidad de municipios distintos:", df["MUNICIPIO"].nunique())
print(df["MUNICIPIO"].sort_values().unique()[:30])  


Cantidad de municipios distintos: 343
['ACATENANGO' 'AGUA BLANCA' 'AGUACATAN' 'ALOTENANGO' 'AMATITLAN'
 'ANTIGUA GUATEMALA' 'ASUNCION MITA' 'ATESCATEMPA' 'AYUTLA' 'BARBERENA'
 'CABANAS' 'CABRICAN' 'CAJOLA' 'CAMOTAN' 'CANILLA' 'CANTEL' 'CASILLAS'
 'CATARINA' 'CHAHAL' 'CHAJUL' 'CHAMPERICO' 'CHIANTLA' 'CHICACAO'
 'CHICAMAN' 'CHICHE' 'CHIMALTENANGO' 'CHINAUTLA' 'CHINIQUE' 'CHIQUIMULA'
 'CHIQUIMULILLA']


### **Ver municipios repetidos escritos diferente dentro de un mismo departamento**

In [16]:
# Agrupar por DEPARTAMENTO y MUNICIPIO
municipios_por_depto = df.groupby(["DEPARTAMENTO", "MUNICIPIO"]).size().reset_index(name="conteo")

# Ver si hay municipios con nombres similares dentro del mismo departamento
print("Total de combinaciones DEPARTAMENTO-MUNICIPIO:", len(municipios_por_depto))


Total de combinaciones DEPARTAMENTO-MUNICIPIO: 348


Se normalizaron los municipios, eliminando tildes y espacios innecesarios. Esto permitió identificar errores ortográficos comunes y garantizar consistencia entre registros. En un análisis más avanzado, se puede validar que cada municipio pertenezca al departamento correcto con una tabla oficial de referencia.

## **ESTABLECIMIENTO**

### **Crear una columna auxiliar ESTABLECIMIENTO_LIMPIO**

Su objetivo es estandarizar nombres para facilitar detección de duplicados y variantes de una misma institución, colegio, entre otros.

In [17]:

def limpiar_nombre_establecimiento(nombre):
    nombre = str(nombre).strip().upper()
    
    # Eliminar todas las comillas dobles y simples en cualquier parte
    nombre = nombre.replace('"', '').replace("'", "")
    
    # Quitar espacios múltiples
    nombre = " ".join(nombre.split())
    
    return nombre

df["ESTABLECIMIENTO_LIMPIO"] = df["ESTABLECIMIENTO"].apply(limpiar_nombre_establecimiento)

print(df[["ESTABLECIMIENTO", "ESTABLECIMIENTO_LIMPIO"]].drop_duplicates().sample(10))

                                        ESTABLECIMIENTO  \
5061                COLEGIO EVANGÉLICO MIXTO "SINAI" II   
3432  INSTITUTO NORMAL MIXTO POR COOPERATIVA ADSCRIT...   
2783  INSTITUTO TÉCNICO VOCACIONAL PRIVADO AMATITLANECO   
6208      INSTITUTO PRIVADO MIXTO "JOSE ERNESTO MONZON"   
1131                     PROGRAMA MODALIDADES FLEXIBLES   
5671                             INSTITUTO "SAN CARLOS"   
6060  CENTRO EDUCATIVO DE FORMACIÓN PROFESIONAL -CEF...   
1688  CENTRO MUNICIPAL DE EDUCACIÓN EXTRAESCOLAR -CEEX-   
2842             CENTRO EDUCATIVO JOSE MILLA Y VIDAURRE   
331            CENTRO EDUCATIVO TECNOLÓGICO EL NAZARENO   

                                 ESTABLECIMIENTO_LIMPIO  
5061                  COLEGIO EVANGÉLICO MIXTO SINAI II  
3432  INSTITUTO NORMAL MIXTO POR COOPERATIVA ADSCRIT...  
2783  INSTITUTO TÉCNICO VOCACIONAL PRIVADO AMATITLANECO  
6208        INSTITUTO PRIVADO MIXTO JOSE ERNESTO MONZON  
1131                     PROGRAMA MODALIDADES FLEXIBLES  
56

### **Visualizar algunos ejemplos de variaciones en nombres de establecimientos para su estandarización**

In [18]:
#%pip install rapid fuzz

In [19]:
from rapidfuzz import process, fuzz

# Obtener todos los nombres únicos
nombres_unicos = df["ESTABLECIMIENTO_LIMPIO"].dropna().unique()

# Diccionario para guardar grupos similares
grupos_similares = {}

# Umbral de similitud 
umbral = 90

# Recorrer nombres y buscar similares
for nombre in nombres_unicos:
    similares = process.extract(nombre, nombres_unicos, scorer=fuzz.token_sort_ratio, limit=10)
    # Filtrar solo los que superan el umbral y no sean el mismo nombre exacto
    similares = [s for s, score, _ in similares if s != nombre and score >= umbral]
    if similares:
        grupos_similares[nombre] = similares

# Mostrar ejemplos de grupos similares encontrados
for i, (base, grupo) in enumerate(grupos_similares.items()):
    print(f"\nGrupo {i+1}:")
    print(f"→ Base: {base}")
    for g in grupo:
        print(f"   - {g}")
    if i == 4:  # Mostrar solo primeros 5 grupos
        break




Grupo 1:
→ Base: ESCUELA NACIONAL DE CIENCIAS COMERCIALES
   - ESCUELA NACIONAL DE CIENCIAS COMERCIALES NO.2
   - ESCUELA NACIONAL DE CIENCIAS COMERCIALES NO.5
   - ESCUELA NACIONAL DE CIENCIAS COMERCIALES NO. 3
   - ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES
   - ESCUELA NACIONAL DE CIENCIAS COMERCIALES AMERICA
   - ESCUELA DE CIENCIAS COMERCIALES NOCTURNA

Grupo 2:
→ Base: INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA
   - INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA
   - INSTITUTO NACIONAL DE EDUCACIÒN DIVERSIFICADA
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADO
   - INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADO
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA INED
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA, INED
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ITZAPA
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA -INED-
   - INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA PUENTE

Grupo 3:
→ Base: COLEGIO CRISTIANO BILINGUE VERBO
   - COLEGIO CRISTIANO BILI

Como se puede observar, si hay algunos nombres de establecimientos que están escritos de diferente manera. Es decir, hay variaciones en cuanto a tíldes, espacios, uso de guiones, entre otros caracteres para representar a un mismo establecimiento, independientemente de si se encuentra en algún otro departamento o municipio. Por ende, se busca generar un csv para poder observar de cerca todas esas variaciones y estandarizar el nombre de los establecimientos.

In [20]:
# Convertir los grupos a lista de pares
pares = []
for base, similares in grupos_similares.items():
    for s in similares:
        pares.append((base, s))

df_similares = pd.DataFrame(pares, columns=["Nombre_Base", "Nombre_Similar"])
df_similares.to_csv("establecimientos_similares.csv", index=False)
print("Archivo exportado como 'establecimientos_similares.csv'")


Archivo exportado como 'establecimientos_similares.csv'


In [21]:

def estandarizar_establecimiento(nombre):
    
    # Reemplazar comillas, comas, guiones innecesarios
    nombre = nombre.replace('"', '').replace("'", '')
    nombre = nombre.replace(',', '').replace('–', '').replace('—', '').replace('-', ' ')
    
    # Corregir NO. separados: NO. 2 -> NO.2
    nombre = re.sub(r"NO\.\s+(\d+)", r"NO.\1", nombre)
    
    # Eliminar espacios extra
    nombre = re.sub(r'\s+', ' ', nombre)

    # Normalizar caracteres con tilde, diéresis, etc.
    nombre = unicodedata.normalize('NFKD', nombre)
    nombre = ''.join(c for c in nombre if not unicodedata.combining(c))

    return nombre

df["ESTABLECIMIENTO_LIMPIO"] = df["ESTABLECIMIENTO"].apply(estandarizar_establecimiento)

Con los patrones observados, se realizó lo siguiente:
- Eliminar espacios extra entre palabras y ó números
- Eliminar acentuación y diéresis para estandarizar nombres
- Eliminar caractéres como ", ', -, entre otros

## **DIRECCIÓN**

### **Limpieza del campo y estandarización de abreviaturas**

In [22]:
def limpiar_direccion(texto):
    if pd.isnull(texto):
        return "SIN INFORMACION"
    
    texto = str(texto).strip().upper()
    
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    
    texto = texto.replace('"', '').replace("'", '').replace('–', '').replace('—', '')
    texto = texto.replace(';', '').replace('.', '').replace(',', '')

    texto = re.sub(r'\bKM\.?(\d)', r'KM \1', texto)

    # Reemplazar abreviaturas comunes
    texto = re.sub(r'\bAV\b|\bAVENIDA\b|\bAVE\b', 'AVENIDA', texto)
    texto = re.sub(r'\bZ\b', 'ZONA', texto)
    texto = re.sub(r'\bCOL\b', 'COLONIA', texto)
    texto = re.sub(r'\s+', ' ', texto)
    
    texto = re.sub(r'\s+', ' ', texto)

    return texto

df["DIRECCION_LIMPIA"] = df["DIRECCION"].apply(limpiar_direccion)


La columna `DIRECCION` representa una de las variables más susceptibles a inconsistencias en el formato, redacción y uso de caracteres especiales, ya que suele ser ingresada manualmente por distintas fuentes. Para garantizar la coherencia y la posibilidad de identificar direcciones duplicadas o similares, se aplicó un proceso de limpieza y estandarización conservador pero completo, implementado en una nueva columna llamada DIRECCION_LIMPIA.

## **TELEFONO**

### **Realizar una limpieza básica de los números telefónicos, tratar valores nulos y estandarizarlos**

In [23]:
def estandarizar_telefono(telefono):
    if pd.isna(telefono):
        return "NO DISPONIBLE"
    
    # Convertir a texto y limpiar espacios
    telefono = str(telefono).strip().upper()
    
    # Si contiene múltiples teléfonos separados por / o ,
    telefono = re.split(r"[\/,]", telefono)[0]
    
    # Eliminar todo lo que no sea dígito
    solo_numeros = re.sub(r"[^\d]", "", telefono)
    
    if not solo_numeros:
        return "NO DISPONIBLE"
    
    if len(solo_numeros) == 8:
        return solo_numeros
    
    if len(solo_numeros) < 8:
        return "FORMATO INCORRECTO: " + telefono
    
    if len(solo_numeros) > 8:
        return solo_numeros[-8:]  # Tomar los últimos 8
    
    return solo_numeros

df["TELEFONO_LIMPIO"] = df["TELEFONO"].apply(estandarizar_telefono)



Se estandarizó el campo `TELEFONO` eliminando todos los caracteres no numéricos y normalizando a un formato de 8 dígitos. 
En caso de múltiples teléfonos, se tomó el primero. Si el número tenía más de 8 dígitos, se conservaron los últimos 8. 
Los campos vacíos o inválidos fueron marcados como `"NO DISPONIBLE"` o `"FORMATO INCORRECTO"`.

## **SUPERVISOR**

### **Limpieza de espacios y acentuaciones en el nombre de los supervisores, además de una validación de posibles valores nulos**

In [24]:
def limpiar_supervisor(nombre):
    if pd.isnull(nombre):
        return "NO REGISTRADO"
    
    
    # Eliminar tildes y caracteres especiales
    nombre = unicodedata.normalize('NFKD', nombre)
    nombre = ''.join(c for c in nombre if not unicodedata.combining(c))
    
    nombre = re.sub(r'\s+', ' ', nombre)

    return nombre

df["SUPERVISOR_LIMPIO"] = df["SUPERVISOR"].apply(limpiar_supervisor)


### **Obtener nombres únicos de cada supervisor**

In [25]:
print("Supervisores distintos:", df["SUPERVISOR_LIMPIO"].nunique())
df["SUPERVISOR_LIMPIO"].value_counts().head(10)


Supervisores distintos: 590


SUPERVISOR_LIMPIO
MIGUEL ANGEL ARMAS ROCHA                          190
CARLOS HUMBERTO GONZALEZ DE LEON                  171
JUAN ENRIQUE MARTINEZ SOLANO                      106
REMY ARTURO SINAY GUDIEL                           94
ELSA YADIRA MARTINEZ PEREZ                         84
MILTON ALONSO ALVAREZ FUENTES                      81
IDALIA DEL ROSARIO LOPEZ SANDOVAL DE PAIZ          81
ELENA ELIZABETH SUCHITE GARNICA DE QUINTANILLA     78
LUDVIN RICARDO URRUTIA LORENTI                     77
JUAN FRANCISCO GODOY DAVILA                        74
Name: count, dtype: int64

### **Aplicar Fuzzy matching para detectar nombres parfecidos y descartar errores tipográficos**

In [26]:

nombres_unicos = df["SUPERVISOR_LIMPIO"].dropna().unique()

grupos_similares = []

for nombre in nombres_unicos:
    similares = process.extract(nombre, nombres_unicos, scorer=fuzz.token_sort_ratio, limit=10)
    for s, score, _ in similares:
        if s != nombre and score >= 90:
            grupos_similares.append((nombre, s, score))

df_fuzzy_supervisores = pd.DataFrame(grupos_similares, columns=["SUPERVISOR_BASE", "SUPERVISOR_SIMILAR", "SIMILITUD"])
df_fuzzy_supervisores = df_fuzzy_supervisores.drop_duplicates(subset=["SUPERVISOR_BASE", "SUPERVISOR_SIMILAR"])


In [27]:
df_fuzzy_supervisores.to_csv("supervisores_similares.csv", index=False)
print("Archivo exportado como 'supervisores_similares.csv'")


Archivo exportado como 'supervisores_similares.csv'


In [28]:
reemplazos_supervisor = {
    "ROERICO DE LA CRUZ ICAL": "RODERICO DE LA CRUZ ICAL"
}

df["SUPERVISOR_LIMPIO"] = df["SUPERVISOR_LIMPIO"].replace(reemplazos_supervisor)


Se creó una nueva columna `SUPERVISOR_LIMPIO` para estandarizar los nombres. Se eliminaron tildes, se convirtió a mayúsculas, y se eliminaron espacios múltiples. Posteriormente se aplicó fuzzy matching para detectar nombres similares escritos de forma distinta 
(por ejemplo: "MARÍA LÓPEZ" vs "MARIA LOPEZ"). Se identificó el nombre de uno de los supervisores con errores tipográficos mediante fuzzy matching. Por ejemplo, ROERICO DE LA CRUZ ICAL tenía una similitud del 97.87% con RODERICO DE LA CRUZ ICAL. Por tanto, se estandarizó utilizando un diccionario de reemplazo.

In [29]:
df.head()

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,...,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,ESTABLECIMIENTO_LIMPIO,DIRECCION_LIMPIA,TELEFONO_LIMPIO,SUPERVISOR_LIMPIO
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,COLEGIO COBAN,KM 2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 2095 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,COLEGIO LA INMACULADA,7A AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,...,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNANDEZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,...,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ,INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...,3A AVENIDA 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNANDEZ


## **DIRECTOR**